In [4]:
# 整合所有鉴定到的subcompartment
import pandas as pd
tissues = ['anther', 'cotyledon', 'ovule', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petal', 'radicle', 'root', 'stem', 'stigma']
new=['Anther', 'Cotyledon', 'Ovule', 'Fiber_10DPA', 'Fiber_20DPA', 'Hypocotyl', 'Leaf', 'Petal', 'Radicle', 'Root', 'Stem', 'Stigma']
old_tag = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
new_tag = ['B1', 'B2', 'B3', 'B4', 'A4', 'A3', 'A2', 'A1']
df_tmp = pd.DataFrame()
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/{}_subcompartment.bed".format(t)
    df = pd.read_csv(input_f, sep="\t", names=['chr', 'start', 'end', 'sub_cpt', 'values', 'zero'], header=None)
    df = df[['chr', 'start', 'end', 'values']].copy()
    df.columns = ['chr', 'start', 'end', t]
    if df_tmp.shape[0] == 0:
        df_tmp = df.copy()
    else:
        df_tmp = pd.merge(df_tmp, df, how='outer', on=['chr', 'start', 'end'])
#df_tmp.dropna(inplace=True)
df_tmp.fillna(0.125, inplace=True)
df = pd.melt(df_tmp, id_vars=('chr', 'start', 'end'))
df.columns = ['chr', 'start', 'end', 'tissues', 'tags']
df['number'] = [1]*df.shape[0]
df['tags'].replace(old_tag, new_tag, inplace=True)
df_statistics = df.groupby(by=['tissues', 'tags'], as_index=False)['number'].sum()
df_statistics['ratio'] = df_statistics['number']/209164
df_statistics['tissues'].replace(tissues, new, inplace=True)
df_statistics.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/statistics/J668_tissues_subcpt_sta.bed", sep="\t", index=False)

In [12]:
# 整合所有鉴定到的subcompartment, 分成At和Dt 两个亚基因组
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_allTissues_subcompartment.bed", sep="\t", names=['chrom', 'start', 'end', 'a', 'tags', 'tissues'])
df['subg'] = df['chrom'].apply(lambda x: 'At' if 'Ghir_A' in x else 'Dt')
# 统计出现的数目
df_sta = pd.DataFrame(df[['tissues', 'subg', 'tags']].value_counts(), columns=['number'])
df_sta.reset_index(inplace=True)
df_sta.sort_values(by=['tissues', 'subg', 'tags'], inplace=True)
df_sta.rename(columns={'tags':'sigs'}, inplace=True)
df_sta.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/statistics/J668_tissues_subcpt_sta_subg.bed", sep="\t", index=False)


In [3]:
# 生成新的 subcompartment格式
## Ghir_A01	1	10000	0.875	A2	A
## Ghir_A01	160001	170000	1.0	A1	A
import pandas as pd
tissues = ['anther', 'cotyledon', 'ovule', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petal', 'radicle', 'root', 'stem', 'stigma']
old_tag = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
new_tag = ['B1', 'B2', 'B3', 'B4', 'A4', 'A3', 'A2', 'A1']
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/{}_subcompartment.bed".format(t)
    output_f = "G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/{}_subcompartment_new.bed".format(t)
    df = pd.read_csv(input_f, sep="\t", names=['chr', 'start', 'end', 'sub_cpt', 'values', 'zero'], header=None)
    df = df[['chr', 'start', 'end', 'values']].copy()
    df['tags'] = df['values']
    df['tags'].replace(old_tag, new_tag, inplace=True)
    df['sig'] = df['tags'].apply(lambda x: 'A' if 'A' in str(x) else 'B')
    df.to_csv(output_f, sep="\t", index=False, header=False)

In [22]:
## 对compartment 调整方向
import pandas as pd
tissues = ['anthers', 'cotyledon', 'ovules', 'fiber_5DPA', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petal', 'radicle', 'root', 'stem', 'stigma']
A_tag = ['A1', 'A2', 'A3', 'A4']
A_tag2 = ['B4', 'B3', 'B2', 'B1']
B_tag = [ 'B1', 'B2', 'B3', 'B4']
B_tag2 = [ 'A4', 'A3', 'A2', 'A1']
old_tag = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
new_tag = ['B1', 'B2', 'B3', 'B4', 'A4', 'A3', 'A2', 'A1']
df_return = pd.DataFrame(columns = ['tissues', 'tags', 'number', 'ratio'])
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/adjust_direact/{}_cpt_adjust.bed".format(t)
    output_f = "G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/adjust_direact/{}_cpt_adjust_T.bed".format(t)
    df = pd.read_csv(input_f, sep=" ", names=['chr', 'start', 'end', 'vaules', 'tags', 'sig1', 'sig2'])
    df_same = df.query("sig1==sig2").copy()
    df_AB = df.query("sig1=='A' & sig2=='B'").copy()
    df_BA = df.query("sig1=='B' & sig2=='A'").copy()
    df_AB['tags'].replace(A_tag, A_tag2, inplace=True)
    df_BA['tags'].replace(B_tag, B_tag2, inplace=True)
    df_out = pd.concat([df_same, df_AB, df_BA], axis=0)
    df_out['vaules'] = df_out['tags']
    df_out['vaules'].replace(new_tag, old_tag, inplace=True)
    df_out.columns = ['chr', 'start', 'end', 'values', 'tags', 'sig1', 'sig2']
    df_out.to_csv(output_f, sep="\t", index=False)
    ## 统计调整方向后的数据比例
    df_out['number'] = [1]*df_out.shape[0]
    df_out1 = df_out.groupby(by=['tags'], as_index=False)['number'].sum()
    df_out1['tissues'] = [t]*df_out1.shape[0]
    df_out1['ratio'] = df_out1['number']/df_out.shape[0]
    #print(df_out1)
    df_return = pd.concat([df_return, df_out1], axis=0)
df_return.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/statistics/J668_tissues_subcpt_sta_adjust.bed", sep="\t", index=False)

In [3]:
## 将所有组织subcompartment整合为一个文件，并且z-score标准化，用umap聚类
import pandas as pd
import scipy.stats as stats
import numpy as np
import umap
from sklearn.preprocessing import StandardScaler
tissues = ['anthers', 'cotyledon', 'ovules', 'fiber_5DPA', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petal', 'radicle', 'root', 'stem', 'stigma']
old_tag = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
new_tag = ['B1', 'B2', 'B3', 'B4', 'A4', 'A3', 'A2', 'A1']
df_tmp = pd.DataFrame()
## 数据整合
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/adjust_direact/{}_cpt_adjust_T.bed".format(t)
    df = pd.read_csv(input_f, sep='\t')
    df = df[['chr', 'start', 'end', 'values']].copy()
    df.columns = ['chr', 'start', 'end', t]
    if df_tmp.shape[0] == 0:
        df_tmp = df.copy()
    else:
        df_tmp = pd.merge(df_tmp, df, how='outer', on=['chr', 'start', 'end'])
## 去除空值
df_tmp.dropna(inplace=True)
df_tmp.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/adjust_direact/J668_subcompartment.bed", sep="\t", index=False)

## z-score 标准化
df_tmp['fiber_20DPA'] = df_tmp['fiber_20DPA']+0.0001 ## 为了防止同一行相同无法标准化
df_tmp['indexs'] = df_tmp['chr'].map(str) + '-' + df_tmp['start'].map(str) + '-' + df_tmp['end'].map(str)
df_tmp = df_tmp[['indexs', 'anthers', 'cotyledon', 'ovules', 'fiber_5DPA', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petal', 'radicle', 'root', 'stem', 'stigma']]
df_index = df_tmp[['indexs']].copy()
df_tmp.set_index("indexs", inplace=True)
df_array = df_tmp.to_numpy()
print(df_array)
df_zscore = stats.zscore(df_array, axis=1, ddof=1)
print(df_zscore)
## 转换为pandas 格式

df_zscore = pd.DataFrame(df_zscore, columns=tissues)

# ## umap 降维处理
# reducer = umap.UMAP()
# data = df_zscore[tissues]
# scaled_penguin_data = StandardScaler().fit_transform(data)
# #print(scaled_penguin_data)
# #df_test = pd.DataFrame(scaled_penguin_data)
# #df_test.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/test.bed", sep="\t", index=False)
# # df_test.drop_duplicates(keep='first', inplace=True)
# # scaled_penguin_data = df_test.to_numpy()
# embedding = reducer.fit_transform(scaled_penguin_data)
# ## 数据整理
# df_index.reset_index(drop=True, inplace=True)
# indexs = df_index['indexs'].tolist()
# df_umap = pd.DataFrame(embedding, columns =['umap1', 'umap2'], index=indexs)
# df_umap.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/umap/J668_subcpt_umap.bed", sep="\t", index=False)

[[1.    1.    1.    ... 1.    1.    0.875]
 [1.    1.    1.    ... 1.    1.    0.875]
 [1.    1.    1.    ... 1.    1.    0.875]
 ...
 [0.625 0.625 0.625 ... 0.625 0.625 0.625]
 [0.875 0.875 0.375 ... 0.75  0.75  0.875]
 [0.875 0.875 0.375 ... 0.75  0.75  0.875]]
[[ 0.27710966  0.27710966  0.27710966 ...  0.27710966  0.27710966
  -3.32820012]
 [ 0.27710966  0.27710966  0.27710966 ...  0.27710966  0.27710966
  -3.32820012]
 [ 0.27710966  0.27710966  0.27710966 ...  0.27710966  0.27710966
  -3.32820012]
 ...
 [-0.2773501  -0.2773501  -0.2773501  ... -0.2773501  -0.2773501
  -0.2773501 ]
 [ 0.70992884  0.70992884 -1.75128907 ...  0.09462436  0.09462436
   0.70992884]
 [ 0.70992884  0.70992884 -1.75128907 ...  0.09462436  0.09462436
   0.70992884]]


In [11]:
import pandas as pd

# 创建示例DataFrame
data = {'Col1': ['A', 'B', 'C', 'A', 'B'],
        'Col2': ['X', 'Y', 'Z', 'X', 'Y']}
df = pd.DataFrame(data)

# 统计两列元素排列组合的次数
combination_counts = df.groupby(['Col1', 'Col2'], set_index=False).size().reset_index(name='Count')

print(combination_counts)

TypeError: groupby() got an unexpected keyword argument 'set_index'

In [3]:
## 将bias 转换为long格式
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_seq/homo_gene2/gene_bias/all_tissue_bias_gene_pair.bed", sep="\t")
df_at = df[['GeneId_At', 'TPM_At', 'sig', 'tissues']].copy()
df_dt = df[['GeneId_Dt', 'TPM_Dt', 'sig', 'tissues']].copy()
df_at.columns = ['GeneId', 'TPM', 'Bias', 'tissues']
df_dt.columns = ['GeneId', 'TPM', 'Bias', 'tissues']
df = pd.concat([df_at, df_dt], axis=0)
df.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_seq/homo_gene2/gene_bias/all_tissue_bias_gene_long.bed", sep="\t", index=False)

In [12]:
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/homo_gene2/gene_bias/all_tissue_bias_gene_pair.bed", sep="\t")
df_result = df.query("(TPM_At/TPM_Dt<=0.5 | TPM_At/TPM_Dt>=2) & (TPM_At>0.5 | TPM_Dt>0.5)")
df_number = df_result.groupby(['tissues', 'sig'], as_index=False).size()


        tissues    sig  size
0        anther  BiasA  3107
1        anther  BiasD  3264
2     cotyledon  BiasA  2757
3     cotyledon  BiasD  2938
4   fiber_10DPA  BiasA  3026
5   fiber_10DPA  BiasD  3259
6   fiber_20DPA  BiasA  2953
7   fiber_20DPA  BiasD  3110
8     hypocotyl  BiasA  2666
9     hypocotyl  BiasD  2799
10         leaf  BiasA  2521
11         leaf  BiasD  2849
12        ovule  BiasA  2256
13        ovule  BiasD  2541
14        petal  BiasA  3399
15        petal  BiasD  3542
16      radicle  BiasA  2449
17      radicle  BiasD  2538
18         root  BiasA  2609
19         root  BiasD  2808
20         stem  BiasA  2719
21         stem  BiasD  2892
22       stigma  BiasA  3424
23       stigma  BiasD  3622


In [15]:
# 分析bias 状态与subcompartment 间的联系
import pandas as pd
df_subcpt = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_all_tissue_gene_subcpt_status.csv", names=['chrom', 'start', 'end', 'GeneId', 'tissues', 'sub_cpt'])
df_bias = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_seq/homo_gene2/gene_bias/all_tissue_bias_gene_long.bed", sep="\t")
df_subcpt = df_subcpt[['GeneId', 'sub_cpt', 'tissues']]
#print(df_subcpt.shape[0])
df_subcpt.drop_duplicates(keep='first', inplace=True, subset=['GeneId', 'tissues'])
df_result = pd.merge(df_bias, df_subcpt, how='inner', on=['GeneId', 'tissues'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_biasAD/J668_subcpt_Bias.bed", sep="\t", index=False)

## 整理为成对的互作,给所有同源基因打上subcpt标签，并整理为成对的格式
df_hg_pair = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_seq/homo_gene2/Ghir_homo_gene_pairs.txt", sep="\t")
tissues = [ 'anther', 'cotyledon', 'ovule',  'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petal', 'radicle', 'root', 'stem', 'stigma']
df_subcpt_bias = pd.DataFrame()
df_hg_bias = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_seq/homo_gene2/gene_bias/all_tissue_bias_gene_pair.bed", sep="\t")
df_hg_bias = df_hg_bias[['GeneId_At', 'GeneId_Dt', 'sig', 'tissues']].copy()
df_hg_bias.columns = ['GeneId_At', 'GeneId_Dt', 'Bias', 'tissues']
for t in tissues:
    df_hg_pair.columns = ['GeneId_At', 'GeneId_Dt']
    df_tissues = df_result.query("tissues==@t")[['GeneId', 'TPM', 'sub_cpt']].copy()
    #print(df_tissues.shape[0])
    df_tissues.columns = ['GeneId_At', 'TPM_At', 'subcpt_At']
    df1 = pd.merge(df_hg_pair, df_tissues, on=['GeneId_At', ])
    df_tissues.columns = ['GeneId_Dt', 'TPM_Dt', 'subcpt_Dt']
    df2 = pd.merge(df1, df_tissues, on=['GeneId_Dt'])
    df2['tissues']  = [t]*df2.shape[0]
    df_subcpt_bias = pd.concat([df_subcpt_bias, df2], axis=0)
print(df_subcpt_bias)
## 筛选能够发生bias表达的同源基因对
df_subcpt_bias = pd.merge(df_subcpt_bias, df_hg_bias, how='inner', on=['GeneId_At', 'GeneId_Dt', 'tissues'])
df_subcpt_bias.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_biasAD/J668_subcpt_Bias_pair.bed", sep="\t", index=False)

            GeneId_At        GeneId_Dt      TPM_At subcpt_At     TPM_Dt  \
0     Ghir_A01G000190  Ghir_D01G000200    1.050841        A1  13.540072   
1     Ghir_A01G000230  Ghir_D01G000240   69.419666        A1  11.728125   
2     Ghir_A01G000240  Ghir_D01G000250    1.207827        A1   0.253538   
3     Ghir_A01G000250  Ghir_D01G000260    0.768050        A1   0.029098   
4     Ghir_A01G000290  Ghir_D01G000300    4.855359        A1   1.555541   
...               ...              ...         ...       ...        ...   
6463  Ghir_A13G024590  Ghir_D13G025370   12.053278        A1  26.018464   
6464  Ghir_A13G024790  Ghir_D13G025560    1.513431        A1   6.099012   
6465  Ghir_A13G024820  Ghir_D13G025590    0.541687        A1   1.434128   
6466  Ghir_A13G024830  Ghir_D13G025600  125.278198        A1  41.888337   
6467  Ghir_A13G024870  Ghir_D13G025640    5.007550        A1   0.379624   

     subcpt_Dt tissues  
0           A1  anther  
1           A1  anther  
2           A1  anther  

In [16]:
## 统计分析与sub-cpt差异相关的同源基因bias 表达
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_biasAD/J668_subcpt_Bias_pair.bed", sep="\t")
df_diffcpt = df.query("subcpt_At != subcpt_Dt").copy()
# 统计subcpt 差异对于同源基因差异影响的数目
df_diffcpt['number'] = [1]*df_diffcpt.shape[0]
df_statistics = df_diffcpt.groupby(by=['tissues'], as_index=False)['number'].sum()
df_statistics.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_biasAD/J668_diffcpt_Bias_number.bed", sep="\t", index=False)


In [69]:
df = df_subcpt_bias.query("tissues=='anthers' & Bias=='BiasD'")
print(df.shape[0])
df1 = df.query("subcpt_At=='A1' & subcpt_Dt!='A1'")
print(df1.shape[0])
df2 = df.query("subcpt_At!='A1' & subcpt_Dt=='A1'")
print(df2.shape[0])

3178
688
208


In [8]:
## 将所有subcompartment 整理为一个文件
import pandas as pd
tissues = [ 'anther', 'cotyledon', 'ovule', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petal', 'radicle', 'root', 'stem', 'stigma']
df_result = pd.DataFrame()
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/{0}_subcompartment_new.bed".format(t)
    df_tmp = pd.read_csv(input_f, sep="\t", names=['chr', 'start', 'end', 'values', 'subcpt', 'cpt'])
    df_tmp['tissues'] = [t]*df_tmp.shape[0]
    df_result = pd.concat([df_result, df_tmp], axis=0)
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_tissues_subcpt.bed", sep="\t", index=False)

In [9]:
## 将所有subcompartment 整理为一个文件， 宽数据格式
import pandas as pd
tissues = [ 'anther', 'cotyledon', 'ovule', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petal', 'radicle', 'root', 'stem', 'stigma']
df_result = pd.DataFrame()
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/{0}_subcompartment_new.bed".format(t)
    df_tmp = pd.read_csv(input_f, sep="\t", names=['chr', 'start', 'end', 'values', 'subcpt', 'cpt'])
    df_tmp = df_tmp[['chr', 'start', 'end', 'subcpt']]
    df_tmp.columns = ['chr', 'start', 'end',  t]
    if df_result.shape[0] == 0:
        df_result = df_tmp.copy()
    else:
        df_result = pd.merge(df_result, df_tmp, how='left', on=['chr', 'start', 'end'])
df_result.dropna(inplace=True)
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_tissues_subcpt_wide.bed", sep="\t", index=False)

In [3]:
## 区分保守区间和动态区间
df_conserved = df_result.query("anthers==cotyledon & cotyledon==ovules & ovules==fiber_10DPA & fiber_10DPA==fiber_20DPA & fiber_20DPA==hypocotyl & hypocotyl==leaf & leaf==petals & petals==radicle & radicle==root & root==stem & stem==stigma").copy()
df_dynamic = df_result.append(df_conserved)
df_dynamic.drop_duplicates(keep=False , inplace=True)
df_conserved.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_subcpt_conserved.bed", sep="\t", index=False)
df_dynamic.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_subcpt_dynamic.bed", sep="\t", index=False)
df_conserved['tags'] = ['conserved']*df_conserved.shape[0]
df_dynamic['tags'] = ['dynamic']*df_dynamic.shape[0]
df_result = pd.concat([df_conserved, df_dynamic], axis=0)
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_subcpt_dynamic_conserved.bed", sep="\t", columns =['chr','start', 'end', 'tags'], index=False)

C:\Users\pc\AppData\Local\Temp\ipykernel_31856\3966958459.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dynamic = df_result.append(df_conserved)


In [1]:
## 将组织特异性基因整合成长数据格式
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/tau_gini/J668_tissue_special_gene.txt", sep="\t")
df_long = pd.melt(df, id_vars=['chr', 'start', 'end', 'GeneId', 'tags'])
df_long.columns = ['chr', 'start', 'end', 'GeneId', 'special_tissues', 'tissues', 'TPM']
df_long.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/tau_gini/J668_tissue_special_gene_long.bed", sep="\t", index=False)
## 将单组织特异性基因选出
tissues = [ 'anthers', 'cotyledon', 'ovules', 'fiber_5DPA', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petals', 'radicle', 'root', 'stem', 'stigma']
for t in tissues:
    #所有组织特异性基因在单一组织中的表达
    # df_tmp = df_long.query("tissues==@t")
    # output_f = "G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/tau_gini/tissues_species_gene/{}_tissue_special_gene.bed".format(t)

    #特定组织特异性基因在所有组织中的表达
    df_tmp = df_long.query("special_tissues==@t")
    output_f = "G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/tau_gini/tissues_species_gene/{}_special_gene.bed".format(t)
    df_tmp.to_csv(output_f, sep="\t", index=False)

In [21]:
## 分析subcompartment 在组织特异性表达基因中的组成
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/tissues/J668_tissues_special_gene_subcpt.bed", sep="\t", names=['chr', 'start', 'end', 'GeneId', 'special_tissues', 'tissues', 'TPM', 'subcpt'])
df['number'] = [1]*df.shape[0]
df.drop_duplicates(keep='first', subset=['GeneId', 'special_tissues', 'tissues'], inplace=True)
df2 = df.groupby(["special_tissues", 'tissues', 'subcpt'], as_index=False)['number'].sum()
df2.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/tissues/J668_tissues_special_gene_subcpt_statistics.bed", sep="\t", index=False)

In [38]:
## 分析组织特异性表达基因是否主要分布在dynamic compartment中
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/DC_subcpt/tissues/J668_tissues_special_gene_DC_subcpt.bed", sep="\t", names=['chr', 'start', 'end', 'GeneId', 'special_tissues', 'tissues', 'TPM', 'DC_tags'])
df['number'] = [1]*df.shape[0]
df_sta = df.groupby(["special_tissues", 'tissues', 'DC_tags'], as_index=False)['number'].sum()
df_sta = df_sta.query("special_tissues==tissues")
df_sta.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/DC_subcpt/tissues/J668_TSG_DC_subcpt_sta.bed", sep="\t", index=False)

In [37]:
## 寻找subcpt 与组织特异性表达相关的例子
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_subcpt_dynamic.bed", sep="\t")
df_anthers = df.query("anthers=='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1' & stigma!='A1'").copy()
df_cotyledon = df.query("anthers!='A1' & cotyledon=='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1' & stigma!='A1'").copy()
df_ovules = df.query("anthers!='A1' & cotyledon!='A1' & ovules=='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_fiber_5DPA = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA=='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1' & stigma!='A1'").copy()
df_fiber_10DPA = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA=='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_fiber_20DPA = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA=='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_hypocotyl = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl=='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_leaf = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf=='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_petals = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals=='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_radicle = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle=='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_root = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root=='A1' & stem!='A1'& stigma!='A1'").copy()
df_stem = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem=='A1'& stigma!='A1'").copy()
df_stigma = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma=='A1'").copy()
print(df_anthers.shape[0], df_cotyledon.shape[0], df_ovules.shape[0], df_fiber_5DPA.shape[0], df_fiber_10DPA.shape[0], df_fiber_20DPA.shape[0], df_hypocotyl.shape[0], df_leaf.shape[0], df_petals.shape[0], df_radicle.shape[0], df_root.shape[0], df_stem.shape[0], df_stigma.shape[0])

df_anthers.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/anthers_A1.bed", sep="\t", index=False, header=False)
df_cotyledon.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/cotyledon_A1.bed", sep="\t", index=False, header=False)
df_ovules.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/ovules_A1.bed", sep="\t", index=False, header=False)
df_fiber_5DPA.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/fiber_5DPA_A1.bed", sep="\t", index=False, header=False)
df_fiber_10DPA.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/fiber_10DPA_A1.bed", sep="\t", index=False, header=False)
df_fiber_20DPA.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/fiber_20DPA_A1.bed", sep="\t", index=False, header=False)
df_hypocotyl.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/hypocotyl_A1.bed", sep="\t", index=False, header=False)
df_leaf.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/leaf_A1.bed", sep="\t", index=False, header=False)
df_petals.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/petals_A1.bed", sep="\t", index=False, header=False)
df_radicle.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/radicle_A1.bed", sep="\t", index=False, header=False)
df_root.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/root_A1.bed", sep="\t", index=False, header=False)
df_stem.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/stem_A1.bed", sep="\t", index=False, header=False)
df_stigma.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/stigma_A1.bed", sep="\t", index=False, header=False)



608 430 970 1146 130 94 264 338 582 1352 1138 98 100


In [40]:
## 寻找subcpt 与组织特异性表达相关的例子, 对Hypocotyl， Stem， Fiber_10DPA, Fiber_20DPA 来找例子
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_subcpt_dynamic.bed", sep="\t")
df_fiber_10DPA = df.query("fiber_10DPA=='A1'").copy()
df_fiber_20DPA = df.query("fiber_20DPA=='A1'").copy()
df_hypocotyl = df.query("hypocotyl=='A1'").copy()
df_stem = df.query("stem=='A1'").copy()

df_fiber_10DPA.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/fiber_10DPA_A1_2.bed", sep="\t", index=False, header=False)
df_fiber_20DPA.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/fiber_20DPA_A1_2.bed", sep="\t", index=False, header=False)
df_hypocotyl.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/hypocotyl_A1_2.bed", sep="\t", index=False, header=False)
df_stem.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/stem_A1_2.bed", sep="\t", index=False, header=False)



In [41]:
## 寻找subcpt 与组织特异性表达相关的例子
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_subcpt_dynamic.bed", sep="\t")
df_fiber_10DPA = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA=='A2' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_fiber_20DPA = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA=='A2' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_hypocotyl = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl=='A2' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem!='A1'& stigma!='A1'").copy()
df_stem = df.query("anthers!='A1' & cotyledon!='A1' & ovules!='A1' & fiber_5DPA!='A1' & fiber_10DPA!='A1' & fiber_20DPA!='A1' & hypocotyl!='A1' & leaf!='A1' & petals!='A1' & radicle!='A1' & root!='A1' & stem=='A2'& stigma!='A1'").copy()

df_fiber_10DPA.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/fiber_10DPA_A2.bed", sep="\t", index=False, header=False)
df_fiber_20DPA.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/fiber_20DPA_A2.bed", sep="\t", index=False, header=False)
df_hypocotyl.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/hypocotyl_A2.bed", sep="\t", index=False, header=False)
df_stem.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/dynamic/stem_A2.bed", sep="\t", index=False, header=False)

In [6]:
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_tpm/J668_gene_subcpt_raw.bed", sep="\t")
df['number'] = [1]*df.shape[0]
df_sta = df.groupby(by=['tissues', 'sub_cpt'], as_index=False)['number'].sum()
df_sta.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_tpm/J668_gene_subcpt_raw_sta.bed", sep="\t", index=False)

In [32]:
## 将subcompartment 所包含的基因整理为宽数据格式
import pandas as pd
tissues = ['anthers', 'cotyledon', 'ovules', 'fiber_5DPA', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'leaf', 'petals', 'radicle', 'root', 'stem', 'stigma']
new_tissues = ['Anther', 'Cotyledon', 'Ovules', 'Fiber_5DPA', 'Fiber_10DPA', 'Fiber_20DPA', 'Hypocotyl', 'Leaf', 'Petals', 'Radicle', 'Root', 'Stem', 'Stigma']
df_gene = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/Gh_gene_position.bed", sep="\t")
df_gene = df_gene[['GeneId']].copy()
df_gene = df_gene.query("~GeneId.str.contains('Scaff')", engine='python').copy()
df_subcpt = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_tpm/J668_gene_subcpt_raw.bed", sep="\t")
df_subcpt.drop_duplicates(keep='first', subset=['GeneId', 'tissues'], inplace=True)
df_result = df_gene.copy()
for t in tissues:
    df_tissues = df_subcpt.query("tissues==@t")[['GeneId', 'sub_cpt']].copy()
    df_tissues.columns = ['GeneId', t]
    df_tmp = pd.merge(df_gene, df_tissues, how='left', on=['GeneId'])
    df_result = pd.merge(df_result, df_tmp, how='left', on=['GeneId'])
#print(df_result)
df_result.columns = ['GeneId', 'Anther', 'Cotyledon', 'Ovule', 'Fiber_5DPA', 'Fiber_10DPA', 'Fiber_20DPA', 'Hypocotyl', 'Leaf', 'Petal', 'Radicle', 'Root', 'Stem', 'Stigma']
df_result.fillna("None", inplace=True)
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_tpm/J668_gene_subcpt_raw_wide.bed", sep="\t", index=False)

In [33]:
## 统计每种不同的subcompartment 所包含的基因数目
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_tpm/J668_gene_subcpt_raw_wide.bed", sep="\t")
df_long = pd.melt(df, id_vars = ("GeneId"))
df_long['number'] = [1]*df_long.shape[0]
df_statics = df_long.groupby(by=['variable', 'value'], as_index=False)['number'].sum()
df_statics.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_tpm/J668_gene_subcpt_raw_wide_sta.bed", sep="\t", index=False)


In [6]:
# 分析动态subcpt 所包含的同源基因对bias 表达，其中active 活性占比
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_biasAD/J668_subcpt_Bias_pair.bed", sep="\t")
old_tags = ['A1', 'A2', 'A3', 'A4', 'B4', 'B3', 'B2', 'B1']
new_tags = [8, 7, 6, 5, 4, 3, 2, 1]
df['subcpt_At'].replace(old_tags, new_tags, inplace=True)
df['subcpt_Dt'].replace(old_tags, new_tags, inplace=True)
df_same = df.query('subcpt_At==subcpt_Dt').copy()
df_Up = df.query("((subcpt_At>subcpt_Dt)&(Bias=='BiasA')) | ((subcpt_Dt>subcpt_At)&(Bias=='BiasD'))").copy()
df_Down = df.query("((subcpt_At>subcpt_Dt)&(Bias=='BiasD')) | ((subcpt_Dt>subcpt_At)&(Bias=='BiasA'))").copy()
print(df_same.shape[0], df_Up.shape[0], df_Down.shape[0], df.shape[0])

37516 13328 12577 63421


In [4]:
# 分析sub-compartment 与WGCNA 间的联系, 分析是否共表达基因所处的染色质活跃程度越高，其共表达的程度越高
import pandas as pd
df_WGCNA = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/WGCNA/J668_all_tissue_005.edges.txt", sep="\t")
df_subcpt = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcpt_tpm/J668_gene_subcpt_raw.bed", sep="\t")
df_subcpt = df_subcpt.query("tissues=='Anther'")[['GeneId', 'sub_cpt']].copy()
df_subcpt.drop_duplicates(keep='first', subset=['GeneId'], inplace=True)
df_subcpt.columns = ['fromNode', 'fromNodeCpt']
df_a = pd.merge(df_WGCNA, df_subcpt, how='left', on=['fromNode'])
df_subcpt.columns = ['toNode', 'toNodeCpt']
df_tmp = pd.merge(df_a, df_subcpt, how='left', on=['toNode'])
df_result = df_tmp.query("fromNodeCpt==toNodeCpt").copy()
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/WGCNA/J668_anther_WGCNA_subcpt.csv", index=False)


In [7]:
## 每组样本随机抽取500个 来分析是否具有显著性差异
sub_cpt = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4']
df_result2 = pd.DataFrame()
for s in sub_cpt:
    df_tmp = df_result.query("fromNodeCpt==@s").copy()
    df_tmp = df_tmp.sample(500)
    df_result2 = pd.concat([df_result2, df_tmp], axis=0)
df_result2.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/WGCNA/J668_anther_WGCNA_subcpt_500.csv", index=False)

In [2]:
import pandas as pd
old_tags = [1, 0.875, 0.75, 0.625, 0.5, 0.375, 0.25, 0.125]
new_tags = ['A1', 'A2', 'A3', 'A4', 'B4', 'B3', 'B2', 'B1']
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_allTissues_subcompartment.bed", sep="\t", names=['chrom', 'start', 'end', 'tags1', 'tags2', 'tissues'])
df['tags2'].replace(old_tags, new_tags, inplace=True)
df.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/compartment/CALDER2/subcompartment/J668_allTissues_subcompartment.bed", sep="\t", index=False, header=False)